In [1]:
import sys
import numpy as np
import pandas as pd
sys.path.append("../code/")

In [2]:
from dataloader import APPLIANCE_ORDER, get_train_test
from tensor_custom_core import stf_4dim, stf_4dim_time

In [3]:
num_folds = 5

In [7]:
def mtf(r=2, lr=1, num_iter=100):
    out = []
    for cur_fold in range(5):
        train, test = get_train_test(num_folds=num_folds, fold_num=cur_fold)
        test_copy = test.copy()
        test_copy[:, 1:, :, :] = np.NaN
        train_test = np.concatenate([train, test_copy])
        H, A, D, T = stf_4dim_time(tensor=train_test,r=r,lr=lr, num_iter=num_iter )
        pred = np.einsum("Hr, Ar, Dr, ATr ->HADT", H, A, D, T)[len(train):, 1:, :, :]
        out.append(pred)
    return np.concatenate(out)


In [10]:
tensor = np.load('../2015-5appliances.numpy.npy')
from sklearn.metrics import mean_absolute_error

err_mtf ={}
gt = tensor[:, 1:, :, :]
for num_latent in range(2, 4):
    print(num_latent)
    pred = mtf(num_latent)
    # Clamping prediction to aggregate
    pred = np.minimum(pred, tensor[:, 0:1, :,:])
    err_mtf[num_latent] = {APPLIANCE_ORDER[i+1]:mean_absolute_error(pred[:, i,:,:].flatten(), 
                                                                       gt[:, i, :, :].flatten()) for i in range(pred.shape[1])}

2
819.868372278 0
400.017517262 50
804.697601901 0
393.038431061 50
807.604072723 0
394.100609765 50
803.448287196 0
379.869636351 50
811.818772316 0
389.034434338 50
3
818.837338412 0
391.602777381 50
803.681772691 0
376.110033224 50
806.580928961 0
382.180544083 50
802.343602488 0
379.195253645 50
810.782270578 0
386.2174 50


In [12]:
pd.DataFrame(err_stf)

,1,2,3
dr,115.048914,115.261059,115.588028
dw,27.510341,27.287977,28.473132
fridge,35.857393,35.967953,36.028281
hvac,254.960051,232.798962,221.404030
mw,10.888660,11.028119,11.014711
